AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [1]:
#%pip install autogluon.timeseries


In [2]:
# 📦 1. Importar librerías
import pandas as pd

In [3]:
# 💬 Instalar AutoGluon si es necesario
#%pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import urllib.request
from pathlib import Path
from typing import Dict, List, Tuple
from datetime import datetime

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna

# Configurar directorios
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("models", exist_ok=True)

In [5]:
CONFIG = {
    "files": {
        "sellin": {
            "local": "sell-in.txt.gz",
            "url": "https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz",
            "read_args": {
                "sep": "\t",
                "compression": "gzip",
                "dtype": {"periodo": str, "customer_id": str, "product_id": str}
            }
        },
        "productos": {
            "local": "tb_productos.txt",
            "url": "https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt",
            "read_args": {
                "sep": "\t",
                "dtype": {"product_id": str}
            }
        },
        "stocks": {
            "local": "tb_stocks.txt",
            "url": "https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt",
            "read_args": {
                "sep": "\t",
                "dtype": {"periodo": str, "product_id": str}
            }
        },
        "productos_a_predecir": {
            "local": "product_id_apredecir201912.txt",
            "url": "https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt",
            "read_args": {
                "sep": "\t",
                "dtype": {"product_id": str}
            }
        }
    },
    "dates": {
        "start": "2017-01-01",
        "end": "2019-12-31"
    }
}

In [6]:
def download_if_needed(local: str, url: str) -> str:
    if not os.path.exists(local):
        print(f"⬇️  Descargando {os.path.basename(local)} ...")
        urllib.request.urlretrieve(url, local)
    return local

def load_raw_files(file_cfg: dict) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Lee sell-in, productos, stocks e ids_a_predecir."""
    paths = {}
    for name, meta in file_cfg.items():
        local_path = os.path.join("data/raw", meta["local"])
        download_if_needed(local_path, meta["url"])
        paths[name] = local_path

    df_sellin = pd.read_csv(paths["sellin"], **file_cfg["sellin"]["read_args"])
    df_productos = pd.read_csv(paths["productos"], **file_cfg["productos"]["read_args"])
    stocks = pd.read_csv(paths["stocks"], **file_cfg["stocks"]["read_args"])
    product_ids = pd.read_csv(paths["productos_a_predecir"], **file_cfg["productos_a_predecir"]["read_args"])

    return df_sellin, df_productos, stocks, product_ids

# Descargar y cargar datos
df_sellin, df_productos, stocks, product_ids = load_raw_files(CONFIG["files"])
print(f"✅ Datos descargados: {len(df_sellin):,} registros de sell-in")

✅ Datos descargados: 2,945,818 registros de sell-in


In [7]:
# 📄 2. Cargar datasets
df_sellin = pd.read_csv("data/raw/sell-in.txt.gz", sep="\t")
df_productos = pd.read_csv("data/raw/tb_productos.txt", sep="\t")

In [8]:
# 📄 Leer lista de productos a predecir
with open("data/raw/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

In [9]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [10]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

if df_filtered.empty:
    print("⚠️ Warning: The filtered DataFrame is empty. Please check the filtering criteria and the input data.")

In [11]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [12]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [13]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [14]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [15]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [16]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60*2)

Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to '/app/AutogluonModels/ag-20250720_054320'
AutoGluon will save models to '/app/AutogluonModels/ag-20250720_054320'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #63~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 22 19:00:15 UTC 2
CPU Count:          8
GPU Count:          1
Memory Avail:       4.15 GB / 23.43 GB (17.7%)
Disk Space Avail:   76.03 GB / 915.32 GB (8.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity': 2}

In [17]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [18]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [19]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [20]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_20_07.csv", index=False)
resultado.head()

,product_id,tn
1,20001,1264.709968
3,20002,1034.922038
5,20003,690.542492
7,20004,504.281123
9,20005,504.275869


In [21]:
# 🔁 Entrenamiento alternativo: predecir diciembre 2019 usando datos hasta 2019-10
df_train_alt = df_sellin[df_sellin['timestamp'] <= '2019-10-01']
df_grouped_alt = df_train_alt.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product_alt = df_grouped_alt.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product_alt['item_id'] = df_monthly_product_alt['product_id']
ts_data_alt = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product_alt,
    id_column='item_id',
    timestamp_column='timestamp'
    )
ts_data_alt = ts_data_alt.fill_missing_values()
predictor_alt = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
    )
predictor_alt.fit(ts_data_alt, num_val_windows=2, time_limit=60*60*2)
forecast_alt = predictor_alt.predict(ts_data_alt)
# Filtrar solo diciembre 2019
resultado_alt = forecast_alt['mean'].reset_index()
resultado_alt = resultado_alt[resultado_alt['timestamp'] == '2019-12-01']
resultado_alt = resultado_alt[['item_id', 'mean']]
resultado_alt.columns = ['product_id', 'tn']
resultado_alt.to_csv("predicciones_diciembre2019_entrenado_hasta_201910.csv", index=False)
resultado_alt.head()

Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to '/app/AutogluonModels/ag-20250720_055911'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #63~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 22 19:00:15 UTC 2
CPU Count:          8
GPU Count:          1
Memory Avail:       4.25 GB / 23.43 GB (18.1%)
Disk Space Avail:   74.70 GB / 915.32 GB (8.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity': 2}

AutoGluon will save models to '/app/AutogluonModels/ag-20250720_055911

,product_id,tn
1,20001,1458.864041
3,20002,1264.308592
5,20003,914.415296
7,20004,765.299802
9,20005,723.793696
